In [132]:
import pandas as pd
import numpy as np
import json
import itertools

In [133]:
path_in_str = 'user_data1.0.json'
json_file = open(path_in_str)
data = json.load(json_file)
df = pd.DataFrame.from_dict(data, orient='index')
df = df[df['error'].isna()]

In [134]:
df['user'] = df.index
df.reset_index(inplace=True)
df['user_id'] = df.index

In [135]:
def melt_series(s):
    lengths = s.str.len().values
    flat = [i for i in itertools.chain.from_iterable(s.values.tolist())]
    idx = np.repeat(s.index.values, lengths)
    return pd.Series(flat, idx, name=s.name)


user_df = melt_series(df.data).to_frame().join(df.drop('data', 1))

In [136]:
user_df['anime_id'] = user_df['data'].apply(lambda x : x['node']['id'])
user_df['status'] = user_df['data'].apply(lambda x : x['list_status']['status'])
user_df['score'] = user_df['data'].apply(lambda x : x['list_status']['score'])
user_df['is_rewatching'] = user_df['data'].apply(lambda x : x['list_status']['is_rewatching'])


In [137]:
user_df = user_df[['user', 'user_id', 'anime_id', 'status', 'score', 'is_rewatching']]
user_df = user_df.rename({'score': 'user_score'}, axis=1)

In [138]:
path_in_str = 'data/anime_list_final_231.json'
json_file = open(path_in_str)
data = json.load(json_file)
anime_df_raw = pd.DataFrame.from_dict(data, orient='index')
anime_df_raw = anime_df_raw[anime_df_raw['error'] != 'not_found']
anime_df = anime_df_raw[['id', 'title', 'mean', 'genres', 'statistics']]
anime_df = anime_df.dropna()

In [139]:
anime_df['genres_name'] = anime_df['genres'].apply(lambda x : [a['name'] for a in x])
anime_df['genres_id'] = anime_df['genres'].apply(lambda x : [a['id'] for a in x])

In [140]:
anime_df['watching'] = anime_df['statistics'].apply(lambda x : x['status']['watching'])
anime_df['num_list_users'] = anime_df['statistics'].apply(lambda x : x['num_list_users'])
anime_df['completed'] = anime_df['statistics'].apply(lambda x : x['status']['completed'])
anime_df['plan_to_watch'] = anime_df['statistics'].apply(lambda x : x['status']['plan_to_watch'])
anime_df['dropped'] = anime_df['statistics'].apply(lambda x : x['status']['dropped'])
anime_df['on_hold'] = anime_df['statistics'].apply(lambda x : x['status']['on_hold'])

In [141]:
anime_df.drop(['genres', 'statistics'], axis=1, inplace=True)

In [144]:
anime_df.rename({'id': 'anime_id'}, axis=1, inplace=True)

In [145]:
anime_df

,anime_id,title,mean,genres_name,genres_id,watching,num_list_users,completed,plan_to_watch,dropped,on_hold
1,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896
6,6.0,Trigun,8.22,"[Action, Adventure, Comedy, Drama, Sci-Fi, Sho...","[1, 2, 4, 8, 24, 27]",33591,624343,375886,171194,15445,28227
8,8.0,Bouken Ou Beet,6.96,"[Adventure, Fantasy, Shounen, Supernatural]","[2, 10, 27, 37]",679,13904,7588,3695,1150,792
16,16.0,Hachimitsu to Clover,8.03,"[Comedy, Drama, Josei, Romance, Slice of Life]","[4, 8, 43, 22, 36]",13026,234999,85960,111377,11903,12733
18,18.0,Initial D Fourth Stage,8.14,"[Action, Cars, Drama, Seinen, Sports]","[1, 3, 8, 42, 30]",7408,140358,107001,20731,1588,3630
...,...,...,...,...,...,...,...,...,...,...,...
49989,49989.0,Yamitsuki Pheromone The Animation,6.50,[Hentai],[12],104,697,281,221,38,53
50029,50029.0,Careless,6.32,[Music],[19],7,213,147,50,5,4
50048,50048.0,Shimi,6.49,"[Music, Romance]","[19, 22]",15,359,209,119,10,6
50058,50058.0,Yellow (Music),7.25,[Music],[19],7,273,226,39,1,0


In [143]:
user_df

,user,user_id,anime_id,status,user_score,is_rewatching
0,chinmay453,0,38101,completed,8,False
0,chinmay453,0,32998,completed,8,False
0,chinmay453,0,34881,completed,7,False
0,chinmay453,0,22199,completed,10,False
0,chinmay453,0,47,completed,0,False
...,...,...,...,...,...,...
95,Kaizuke,95,32615,plan_to_watch,0,False
95,Kaizuke,95,35507,completed,10,False
95,Kaizuke,95,34798,plan_to_watch,0,False
95,Kaizuke,95,23283,completed,10,False


In [146]:
df_merge = pd.merge(anime_df, user_df, on = 'anime_id')

In [147]:
df_merge

,anime_id,title,mean,genres_name,genres_id,watching,num_list_users,completed,plan_to_watch,dropped,on_hold,user,user_id,status,user_score,is_rewatching
0,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896,ShinyHex,1,plan_to_watch,0,False
1,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896,illegalarief,2,completed,7,False
2,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896,Zura15,3,completed,10,False
3,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896,Azathanai,5,completed,8,False
4,1.0,Cowboy Bebop,8.77,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]","[1, 2, 4, 8, 24, 29]",137020,1488371,837883,397985,31587,83896,GalacticAdvent,6,completed,10,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49725,49738.0,Heike Monogatari,7.62,[Historical],[13],20011,38643,0,16230,1416,986,Kapusta_Put,76,watching,0,False
49726,49738.0,Heike Monogatari,7.62,[Historical],[13],20011,38643,0,16230,1416,986,Thorf,77,dropped,0,False
49727,49738.0,Heike Monogatari,7.62,[Historical],[13],20011,38643,0,16230,1416,986,Kuraya,82,plan_to_watch,0,False
49728,49738.0,Heike Monogatari,7.62,[Historical],[13],20011,38643,0,16230,1416,986,Sudhanshu_CH,86,plan_to_watch,0,False
